In [15]:
import requests
import numpy as np
import pandas as pd
pd.set_option("display.max_columns", None)
import time
import sklearn
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from datetime import datetime

In [16]:
# Date format : year, month, day

date = "2024-09-13"

In [17]:
def scale_datasets(x_train):
    standard_scaler = StandardScaler()
    x_train_scaled = pd.DataFrame(standard_scaler.fit_transform(x_train),columns=x_train.columns)
    return x_train_scaled

In [18]:
data_original = pd.read_csv("dataset_alpha_4_1_for_neuralnetwork_total.csv")
print("dataset_alpha_2_1_for_neuralnetwork done")


y_real1 = pd.read_csv("y_real1_wparam_alpha1_1_for_function.csv")
print("y_real1_wparam_alpha1_1_for_function done")

y_real2 = pd.read_csv("y_real2_wparam_alpha1_1_for_function.csv")
print("y_real2_wparam_alpha1_1_for_function done")

y_real3 = pd.read_csv("y_real3_wparam_alpha1_1_for_function.csv")
print("y_real3_wparam_alpha1_1_for_function done")

y_real4 = pd.read_csv("y_real4_wparam_alpha1_1_for_function.csv")
print("y_real4_wparam_alpha1_1_for_function done")

y_real5 = pd.read_csv("y_real5_wparam_alpha1_1_for_function.csv")
print("y_real5_wparam_alpha1_1_for_function done")

y_real6 = pd.read_csv("y_real6_wparam_alpha1_1_for_function.csv")
print("y_real6_wparam_alpha1_1_for_function done")

y_real7 = pd.read_csv("y_real7_wparam_alpha1_1_for_function.csv")
print("y_real7_wparam_alpha1_1_for_function done")

y_real8 = pd.read_csv("y_real8_wparam_alpha1_1_for_function.csv")
print("y_real8_wparam_alpha1_1_for_function done")





y_real_original = pd.concat([y_real1,y_real2,y_real3,y_real4,y_real5,y_real6,y_real7,y_real8],axis=0)



data = data_original.copy()
y_real = y_real_original.copy()
print(data.shape, y_real.shape)


data.drop(columns=["Unnamed: 0"],inplace=True)
data.drop(columns=["Unnamed: 0.1"],inplace=True)
data.drop(columns=["Unnamed: 0.2"],inplace=True)

columns_to_remove = ['dogId1', 'dogId2', 'dogId3', 'dogId4', 'dogId5', 'dogId6']

# Remove the columns
data = data.drop(columns=columns_to_remove)

columns_to_remove = ['raceClass']

# Remove the columns
data = data.drop(columns=columns_to_remove)


#one hot encode traps_number #

from sklearn.preprocessing import OneHotEncoder

ohe = OneHotEncoder(sparse_output=False)
traps_number_encoded = ohe.fit_transform(y_real)

traps_final = pd.DataFrame(traps_number_encoded, columns=ohe.get_feature_names_out([str(col) for col in y_real.columns]))


y_real = traps_final.copy()


def remove_columns_with_too_many_zeros_1(df, threshold=0.7):
    # Initialize an empty list to store columns to drop
    columns_to_drop = []
    
    # Loop through the generated column names
    for j in range(1, 7):
        for i in range(1, 7):
            column_name = f"traps_number{i}percentagedog{j}"
            if column_name in df.columns:
                # Calculate the percentage of zeros in the column
                zero_percentage = (df[column_name] == 0).mean()
                # If the percentage of zeros is greater than the threshold, mark for dropping
                if zero_percentage > threshold:
                    print("the column dropped is : ", column_name)
                    columns_to_drop.append(column_name)
    
    # Drop the identified columns
    df_dropped = df.drop(columns=columns_to_drop)
    
    return df_dropped

# Apply the function to the DataFrame
data = remove_columns_with_too_many_zeros_1(data)


def remove_columns_with_too_many_zeros_2(df, threshold=0.7):
    # Initialize an empty list to store columns to drop
    columns_to_drop = []
    
    # Loop through the generated column names
    for j in range(1, 6):  # race_distance_number{j} (1 to 5)
        for i in range(1, 7):  # percentagedog{i} (1 to 6)
            column_name = f"race_distance_number{j}percentagedog{i}"
            if column_name in df.columns:
                # Calculate the percentage of zeros in the column
                zero_percentage = (df[column_name] == 0).mean()
                # If the percentage of zeros is greater than the threshold, mark for dropping
                if zero_percentage > threshold:
                    print("the column dropped is : ", column_name)
                    columns_to_drop.append(column_name)
    
    # Drop the identified columns
    df_dropped = df.drop(columns=columns_to_drop)
    
    return df_dropped

# Apply the function to the DataFrame
data = remove_columns_with_too_many_zeros_2(data)


substrings_to_remove = [
    'markov_probability6',
    'markov_probability5',
    'markov_probability4',
    'markov_probability3', 'markov_probability2', 'markov_probability1', 
    'SP1', 'SP2', 'SP3', 'SP4', 'SP5', 'SP6', 'resultMarketPos1', 'resultMarketPos2', 'resultMarketPos3', 
    'resultMarketPos4', 'resultMarketPos5', 'resultMarketPos6'
]

# Create a regex pattern that matches any of the substrings
pattern = '|'.join(substrings_to_remove)

# Use the filter method with regex to get a list of columns to keep
columns_to_keep = data.columns[~data.columns.str.contains(pattern)]

# Create a new dataframe with only the columns to keep
data = data[columns_to_keep]


# Added : 
data =  data.drop([col for col in data.columns if 'TotAvgPosition' in col], axis=1)
data =  data.drop([col for col in data.columns if 'AvgPosition5_' in col], axis=1)
data =  data.drop([col for col in data.columns if 'AvgRaceRemarks5_' in col], axis=1)
data =  data.drop([col for col in data.columns if 'TotAvgRaceRemarks' in col], axis=1)
data =  data.drop([col for col in data.columns if 'DistanceToFirstAvg_' in col], axis=1)
data =  data.drop([col for col in data.columns if 'DistanceToFirstAvg5_' in col], axis=1)



data.fillna(data.mean(), inplace=True)


## check empty columns ##
columns_with_nan = data.columns[data.isna().any()].tolist()
print(columns_with_nan)


print(data.shape)
print(y_real.shape)

contains_infinity = np.isinf(data).values.any()

# Count the number of occurrences of infinity
num_infinity = np.isinf(data).sum().sum()

print("Does the DataFrame contain infinity values?", contains_infinity)
print("Number of occurrences of infinity values:", num_infinity)

for i in range(1,7):
    data.drop(columns=[f"DogsectionalTimelastraces{i}"],inplace=True)


Column_names_for_prediction_today = data.columns

standard_scaler = StandardScaler()
standard_scaler.fit(data)

dataset_alpha_2_1_for_neuralnetwork done
y_real1_wparam_alpha1_1_for_function done
y_real2_wparam_alpha1_1_for_function done
y_real3_wparam_alpha1_1_for_function done
y_real4_wparam_alpha1_1_for_function done
y_real5_wparam_alpha1_1_for_function done
y_real6_wparam_alpha1_1_for_function done
y_real7_wparam_alpha1_1_for_function done
y_real8_wparam_alpha1_1_for_function done
(154760, 287) (154760, 6)
the column dropped is :  traps_number5percentagedog1
the column dropped is :  traps_number6percentagedog1
the column dropped is :  traps_number5percentagedog2
the column dropped is :  traps_number6percentagedog2
the column dropped is :  traps_number6percentagedog3
the column dropped is :  traps_number1percentagedog4
the column dropped is :  traps_number1percentagedog5
the column dropped is :  traps_number2percentagedog5
the column dropped is :  traps_number1percentagedog6
the column dropped is :  traps_number2percentagedog6
the column dropped is :  traps_number3percentagedog6
the column dro

StandardScaler()

In [19]:
data_to_predict= pd.read_csv(f"final_data_analysis_results_new_prediction{date}.csv")
print("import final_data_analysis_results_new_prediction done")


data_to_predict = data_to_predict[Column_names_for_prediction_today]

mean_values = data.mean()

# Fill missing values in `df_data_bet` with the corresponding mean values from `df_data`

#data_to_predict = data_to_predict.fillna(mean_values) ##############


#data_to_predict.fillna(mean_values, inplace=True)

## check empty columns ##
columns_with_nan = data_to_predict.columns[data_to_predict.isna().any()].tolist()
print("the column with nan values are : ")
print(columns_with_nan)

print(data_to_predict.shape)


contains_infinity = np.isinf(data_to_predict).values.any()

# Count the number of occurrences of infinity
num_infinity = np.isinf(data_to_predict).sum().sum()

print("Does the DataFrame contain infinity values?", contains_infinity)
print("Number of occurrences of infinity values:", num_infinity)


data_to_predict = pd.DataFrame(standard_scaler.transform(data_to_predict), columns=data_to_predict.columns)

import final_data_analysis_results_new_prediction done
the column with nan values are : 
['dogBorn4', 'dogBorn5']
(42, 152)
Does the DataFrame contain infinity values? False
Number of occurrences of infinity values: 0


In [20]:
race_infos = pd.read_csv(f"races_track_and_hour{date}.csv")

In [21]:
nan_indices = data_to_predict[data_to_predict.isna().any(axis=1)].index

data_to_predict = data_to_predict.drop(nan_indices)
race_infos = race_infos.drop(nan_indices)

data_to_predict = data_to_predict.reset_index(drop=True) 
race_infos = race_infos.reset_index(drop=True)

In [22]:
data = scale_datasets(data)
y_real = scale_datasets(y_real)


X_train_val, X_test, y_train_val, y_test = train_test_split(data, y_real,
    test_size=0.2, shuffle = True, random_state = 13)

X_train, X_val, y_train, y_val = train_test_split(X_train_val, y_train_val,
    test_size=0.1, shuffle = True, random_state = 12)
X_train.shape, X_val.shape, y_train.shape, y_val.shape, X_test.shape, y_test.shape



import tensorflow as tf
from tensorflow.keras import layers
from tensorflow.keras.regularizers import l2
callback = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=20,restore_best_weights=True)


## best model ##
model1 = tf.keras.Sequential([
      layers.Dense(50, activation='relu',input_shape=(data.shape[1],)),
    layers.BatchNormalization(),
    layers.Dense(100),
    layers.BatchNormalization(),
    layers.Dropout(0.4),
    layers.Dense(50),
    layers.BatchNormalization(),
      layers.Dense(y_real.shape[1])
  ])

model1.compile(loss=tf.keras.losses.MeanSquaredError(),
                optimizer=tf.keras.optimizers.Adam(0.006),metrics=["accuracy"])  


model1.fit(X_train,y_train,batch_size=300,epochs=400,validation_data=(X_val, y_val),callbacks=[callback])




Epoch 1/400


/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


372/372 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - accuracy: 0.0313 - loss: 1.0718 - val_accuracy: 0.0270 - val_loss: 1.0036
Epoch 2/400
372/372 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.0509 - loss: 1.0014 - val_accuracy: 0.0687 - val_loss: 0.9955
Epoch 3/400
372/372 ━━━━━━━━━━━━━━━━━━━━ 0s 997us/step - accuracy: 0.0743 - loss: 0.9955 - val_accuracy: 0.0768 - val_loss: 0.9940
Epoch 4/400
372/372 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.0813 - loss: 0.9931 - val_accuracy: 0.0892 - val_loss: 0.9920
Epoch 5/400
372/372 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.0870 - loss: 0.9914 - val_accuracy: 0.0764 - val_loss: 0.9917
Epoch 6/400
372/372 ━━━━━━━━━━━━━━━━━━━━ 0s 999us/step - accuracy: 0.0852 - loss: 0.9903 - val_accuracy: 0.0871 - val_loss: 0.9912
Epoch 7/400
372/372 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.0877 - loss: 0.9900 - val_accuracy: 0.0897 - val_loss: 0.9910
Epoch 8/400
372/372 ━━━━━━━━━━━━━━━━━━━━ 0s 994us/step - accuracy: 0.0868 - loss: 0.9892 - val_accuracy: 

In [23]:
def get_result_market_pos(final_data):
    for j in range(6):
        loc = final_data.columns.get_loc(f'SP{j+1}') + 1
        final_data.insert(loc, f'resultMarketPos{j+1}', np.nan)
    for i in range(len(final_data)):
        dogSP = []
        for j in range(1,7):
            SP = final_data[f"SP{j}"][i]
            dogSP.append(SP)
        indices_initiaux = list(range(len(dogSP)))
        indices_tries = sorted(indices_initiaux, key=lambda x: dogSP[x])
        positions = [0] * len(dogSP)
        
        for rang, indice in enumerate(indices_tries):
            positions[indice] = rang
        #print(len(positions))
        #print(len(dogSP))
        if len(dogSP) == 6:
            for k in range(6):
                final_data[f"resultMarketPos{k+1}"][i] = positions[k]+1

    return final_data

In [24]:
def scale_datasets(x_train):
    standard_scaler = StandardScaler()
    x_train_scaled = pd.DataFrame(standard_scaler.fit_transform(x_train),columns=x_train.columns)
    return x_train_scaled

In [25]:
def pred_one_hot_encoding(prediction):
    positions = [0,0,0,0,0,0]
    pred = prediction.reshape(6,6)
    pred = pd.DataFrame(pred)
    for i in range(6):
        chunk = pred[i]
        max_index_i = chunk.idxmax()
        positions[i] = max_index_i+1
        pred = pred.drop(index=max_index_i)
    return positions

In [26]:
def find_confidance_pred1(pred):
    diff  = [0,0,0,0,0,0]
    pred = pd.DataFrame(pred.reshape(6,6))
    pos1 = pred[0]
    max_index = pos1.idxmax()
    val = pos1[max_index]
    for i in range(6):
        conf = val - pos1[i]
        diff[i] = conf
    diff = [x for x in diff if x != 0]
    x = min(diff)
    return x

In [27]:
## find precision directly from NN models ##
def precision_one_hot_data_trained(X_test,y_test,model):
    val_pred = model.predict(X_test)
    percents = [0,0,0,0,0,0]
    for i in range(len(y_test)):
        real = y_test.iloc[i]
        real = real.to_numpy()
        real = pred_one_hot_encoding(real)
        pred = val_pred[i]
        pred = pred_one_hot_encoding(pred)
    
        for j in range(6):
            if pred[j] == real[j]:
                percents[j] += 1            
    percents = [x/len(y_test)*100 for x in percents]
    return percents


print(precision_one_hot_data_trained(X_test,y_test,model1))


968/968 ━━━━━━━━━━━━━━━━━━━━ 0s 218us/step
[23.478288963556473, 19.753166192814682, 17.540062031532695, 17.598216593434994, 19.010080124063062, 17.31390540191264]


In [28]:
data_to_predict.shape,race_infos.shape

((37, 152), (37, 2))

In [29]:
data_bet = data_to_predict

In [30]:
## check empty columns ##
columns_with_nan = data_bet.columns[data_bet.isna().any()].tolist()
print(columns_with_nan)

[]


In [31]:
race_infos.shape,data_bet.shape

((37, 2), (37, 152))

In [32]:
#data_bet = data_bet[data.columns]
#standard_scaler = StandardScaler()
#standard_scaler.fit(data)
data_scaled = data_bet
#data_scaled = pd.DataFrame(standard_scaler.transform(data_bet),columns=data_bet.columns)
x = data_scaled.iloc[0].to_numpy().reshape(1,-1)
#race = pd.DataFrame(x,columns=data_bet.columns)
pred = model1.predict(data_scaled)
pred2 = model1.predict(x)
#p = pred_one_hot_encoding(pred)
pred2

2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 899us/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step


array([[ 0.03585781, -0.10508288, -0.02256801, -0.01013453,  0.0475652 ,
         0.0651853 , -0.2768472 , -0.22988495, -0.0899606 ,  0.07397456,
         0.2190089 ,  0.3101512 , -0.25571722, -0.16448133, -0.15021133,
         0.0136363 ,  0.14218284,  0.41877714, -0.01420277,  0.06739251,
         0.07829994, -0.0171665 , -0.03000412, -0.08402996,  0.29418972,
         0.21721774,  0.10335387, -0.07391046, -0.19793107, -0.32430154,
         0.24199429,  0.21400653,  0.09857018,  0.01095819, -0.18792489,
        -0.38578865]], dtype=float32)

In [33]:
z = pred_one_hot_encoding(pred2)
z

[5, 6, 4, 2, 3, 1]

In [34]:
x = data_scaled.iloc[0].to_numpy().reshape(1,-1)
race = pd.DataFrame(x,columns=data_bet.columns)
race.shape
#x.shape

(1, 152)

In [35]:
race_infos.head()

,raceTrack,raceTimeDate
0,CENTRAL PARK,2024-09-13 12:51
1,HARLOW,2024-09-13 11:04
2,HARLOW,2024-09-13 13:03
3,HOVE,2024-09-13 11:01
4,HOVE,2024-09-13 11:34


In [36]:
print("ALL RACES:")
usefull_infos = race_infos[["raceTrack","raceTimeDate"]]
tot_predictions = []
for i in range(len(pred)):
    p = pred_one_hot_encoding(pred[i])
    infos = usefull_infos.iloc[i].to_numpy()
    conf = find_confidance_pred1(pred[i])
    conf = np.round(conf,2)
    print(f"Race infos:{infos}, Prediction:{p}, Confidance:{conf}")

ALL RACES:
Race infos:['CENTRAL PARK' '2024-09-13 12:51'], Prediction:[5, 6, 4, 2, 3, 1], Confidance:0.05000000074505806
Race infos:['HARLOW' '2024-09-13 11:04'], Prediction:[5, 2, 3, 4, 6, 1], Confidance:0.15000000596046448
Race infos:['HARLOW' '2024-09-13 13:03'], Prediction:[1, 3, 5, 4, 2, 6], Confidance:0.20999999344348907
Race infos:['HOVE' '2024-09-13 11:01'], Prediction:[1, 5, 2, 3, 4, 6], Confidance:0.33000001311302185
Race infos:['HOVE' '2024-09-13 11:34'], Prediction:[5, 3, 4, 2, 6, 1], Confidance:0.14000000059604645
Race infos:['HOVE' '2024-09-13 13:51'], Prediction:[4, 6, 2, 1, 5, 3], Confidance:0.03999999910593033
Race infos:['HOVE' '2024-09-13 14:07'], Prediction:[3, 5, 6, 4, 2, 1], Confidance:0.07999999821186066
Race infos:['KINSLEY' '2024-09-13 14:29'], Prediction:[6, 1, 2, 3, 5, 4], Confidance:0.03999999910593033
Race infos:['KINSLEY' '2024-09-13 15:26'], Prediction:[6, 4, 1, 2, 3, 5], Confidance:0.2800000011920929
Race infos:['KINSLEY' '2024-09-13 15:44'], Prediction:

In [37]:
array_of_prediction = []

usefull_infos = race_infos[["raceTrack","raceTimeDate"]]
tot_predictions = []
for i in range(len(pred)):
    p = pred_one_hot_encoding(pred[i])
    infos = usefull_infos.iloc[i].to_numpy()

    array_of_prediction.append(p[0])

print(array_of_prediction)

[5, 5, 1, 1, 5, 4, 3, 6, 6, 5, 2, 1, 4, 5, 6, 3, 2, 1, 5, 3, 3, 3, 6, 3, 2, 5, 5, 4, 6, 4, 5, 5, 3, 2, 1, 4, 5]


In [38]:
races_track_and_hour = race_infos.copy()
races_track_and_hour['result'] = None
races_track_and_hour['Official_result'] = None



for position in range(len(races_track_and_hour)): 
    races_track_and_hour.at[position, "result"] = array_of_prediction[position]


races_track_and_hour


,raceTrack,raceTimeDate,result,Official_result
0,CENTRAL PARK,2024-09-13 12:51,5,None
1,HARLOW,2024-09-13 11:04,5,None
2,HARLOW,2024-09-13 13:03,1,None
3,HOVE,2024-09-13 11:01,1,None
4,HOVE,2024-09-13 11:34,5,None
5,HOVE,2024-09-13 13:51,4,None
6,HOVE,2024-09-13 14:07,3,None
7,KINSLEY,2024-09-13 14:29,6,None
8,KINSLEY,2024-09-13 15:26,6,None
9,KINSLEY,2024-09-13 15:44,5,None


In [39]:
races_track_and_hour.to_csv("races_track_and_hour_to_convert_to_pdf.csv", index=False)